In [1]:
######### 분석과제!
rm(list=ls())
### setwd

In [2]:
### packages
if(!require(caret)) install.packages("caret"); library(caret)
if(!require(e1071)) install.packages("e1071"); library(e1071)
if(!require(dplyr)) install.packages("dplyr"); library(dplyr)
if(!require(class)) install.packages("class"); library(class) # for knn
if(!require(kknn)) install.packages("kknn"); library(kknn) # for wknn
if(!require(data.table)) install.packages("data.table"); library(data.table)

Loading required package: caret
Loading required package: lattice
Loading required package: ggplot2
Loading required package: e1071
Loading required package: dplyr

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: class
Loading required package: kknn

Attaching package: 'kknn'

The following object is masked from 'package:caret':

    contr.dummy

Loading required package: data.table

Attaching package: 'data.table'

The following objects are masked from 'package:dplyr':

    between, first, last



In [3]:
#### load data
pro.train <- fread('data/profiles_train.csv')
pro.train <- as.data.frame(pro.train)
pro.train$gen <- as.factor(pro.train$gen)
click.train <- fread('data/click_train.csv')
click.train <- as.data.frame(click.train)
pro.test <- fread('data/profiles_test.csv')
pro.test <- as.data.frame(pro.test)
click.test <- fread('data/click_test.csv')
click.test <- as.data.frame(click.test)
pro.train %>% head
click.train %>% head
str(click.train)

id,gen,job,resid
1003,여자,의료,서울
1006,남자,자영업,대전
1014,남자,자영업,경상
1025,여자,대학/대학원생,부산
1027,남자,전산직,경기
1039,남자,사무관리직,경기


id,time,st,st_c,st_t,cate
1003,2012072311,up.cafe.naver.com,1,0,커뮤니티
1003,2012080211,blog.naver.com,11,26,커뮤니티
1003,2012080712,cafe.naver.com,1,60,커뮤니티
1003,2012081011,blog.naver.com,59,346,커뮤니티
1003,2012082713,blog.daum.net,10,532,커뮤니티
1003,2012090711,blog.naver.com,39,93,커뮤니티


'data.frame':	1393986 obs. of  6 variables:
 $ id  : chr  "1003" "1003" "1003" "1003" ...
 $ time: int  2012072311 2012080211 2012080712 2012081011 2012082713 2012090711 2012091710 2012110615 2012110617 2012111012 ...
 $ st  : chr  "up.cafe.naver.com" "blog.naver.com" "cafe.naver.com" "blog.naver.com" ...
 $ st_c: int  1 11 1 59 10 39 8 1 1 7 ...
 $ st_t: int  0 26 60 346 532 93 12 53 60 805 ...
 $ cate: chr  "커뮤니티" "커뮤니티" "커뮤니티" "커뮤니티" ...


In [4]:
click.train$time = as.character(click.train$time)
click.train$time = strtrim(click.train$time,8)
click.train$time = as.numeric(click.train$time)
click.test$time = as.character(click.test$time)
click.test$time = strtrim(click.test$time,8)
click.test$time = as.numeric(click.test$time)

In [5]:
d <- click.train %>% group_by(id) %>% summarise(Day = length(unique(time)))
a <- click.train %>% group_by(id) %>% summarise(ST_T = sum(st_t)) # 총 체류시간
b <- click.train %>% group_by(id) %>% summarise(ST_C = sum(st_c)) # 총 페이지본 횟수
c <- click.train %>% group_by(id) %>% summarise(CATE = length(unique(cate)))


In [6]:
pro.train$st_t = as.numeric(a$ST_T)
pro.train$st_c = as.numeric(b$ST_C)
pro.train$cov = as.numeric(c$CATE/22)
pro.train$day = as.numeric(d$Day)
pro.train = within(pro.train, rm("job","resid", "id"))

In [7]:
pro.train %>% str()

'data.frame':	402 obs. of  5 variables:
 $ gen : Factor w/ 2 levels "남자","여자": 2 1 1 2 1 1 2 1 1 1 ...
 $ st_t: num  885021 138740 337567 426651 354698 ...
 $ st_c: num  34024 10651 9288 28349 12539 ...
 $ cov : num  0.773 0.818 0.864 0.864 0.818 ...
 $ day : num  129 129 185 107 194 223 130 166 150 126 ...


In [8]:
d <- click.test %>% group_by(id) %>% summarise(Day = length(unique(time)))
a <- click.test %>% group_by(id) %>% summarise(ST_T = sum(st_t)) # 총 체류시간
b <- click.test %>% group_by(id) %>% summarise(ST_C = sum(st_c)) # 총 페이지본 횟수
c <- click.test %>% group_by(id) %>% summarise(CATE = length(unique(cate)))
# pro.test$gen = as.numeric(pro.test$gen)-1
pro.test$st_t = as.numeric(a$ST_T)
pro.test$st_c = as.numeric(b$ST_C)
pro.test$cov = as.numeric(c$CATE/22)
pro.test$day = as.numeric(d$Day)
pro.test = within(pro.test, rm("job","resid", "id"))
pro.test %>% str()

'data.frame':	99 obs. of  4 variables:
 $ st_t: num  230324 161998 137625 474261 730673 ...
 $ st_c: num  10102 8368 6750 16146 29484 ...
 $ cov : num  0.818 0.591 0.682 0.909 0.909 ...
 $ day : num  113 147 197 111 123 117 108 113 213 113 ...


In [10]:
## min-max 스케일링
normalize <- function(x){
  return( (x-min(x))/(max(x)-min(x)) )
}

In [14]:
pro.train.normal <- as.data.frame(lapply(pro.train[-1], normalize))
pro.test.normal <- as.data.frame(lapply(pro.test, normalize))


In [15]:
pro.train.normal$gen = pro.train$gen
pro.train.normal$cov = pro.train$cov
pro.test.normal$cov = pro.test$cov

In [16]:
str(pro.train.normal)

'data.frame':	402 obs. of  5 variables:
 $ st_t: num  0.2044 0.0302 0.0766 0.0974 0.0806 ...
 $ st_c: num  0.412 0.122 0.105 0.342 0.145 ...
 $ cov : num  0.773 0.818 0.864 0.864 0.818 ...
 $ day : num  0.195 0.195 0.414 0.109 0.449 ...
 $ gen : Factor w/ 2 levels "남자","여자": 2 1 1 2 1 1 2 1 1 1 ...


In [17]:
pro.train.normal %>% head
pro.test.normal %>% head

st_t,st_c,cov,day,gen
0.20435473,0.4120312,0.7727273,0.1953125,여자
0.03024735,0.1219500,0.8181818,0.1953125,남자
0.07663370,0.1050339,0.8636364,0.4140625,남자
0.09741700,0.3415990,0.8636364,0.1093750,여자
0.08063036,0.1453819,0.8181818,0.4492188,남자
0.28894979,0.4882344,1.0000000,0.5625000,남자


st_t,st_c,cov,day
0.08878565,0.11385648,0.8181818,0.07608696
0.05415057,0.08897594,0.5909091,0.26086957
0.04179568,0.06575983,0.6818182,0.53260870
0.21243955,0.20057969,0.9090909,0.06521739
0.34241715,0.39196189,0.9090909,0.13043478
0.07811622,0.28124776,0.5909091,0.09782609


In [81]:
set.seed(1)
# createDataPartition
# caret패키지 안에 있는 데이터 파티션, 층화추출
idx <- createDataPartition(y = pro.train.normal$gen, p = 0.7, list =FALSE)
train1 <- pro.train.normal[idx,]
test1 <- pro.train.normal[-idx,]
### train 내에서 분할하고 kknn, glm, lda, knn 이 있는데 glm과 lda 가 완전히 똑같이 분류하여
### vote에 많은 영향을 주어 lda 와 glm 중 하나만 넣는 것이 좋을 듯 합니다.

In [35]:
repCv <- trainControl(method = "repeatedcv", number = 6,repeats = 4, verbose = T)
cv <- trainControl(method = "cv", number = 5, verbose = T)
knn.grid = expand.grid(
  .k = c(1,3,5,7,9)
)
wknn.grid = expand.grid(
  .kmax = c(1,3,5,7,9),
  .distance = c(1,2),
  .kernel = "optimal"
)

In [60]:
voteClassifier <- function() {
  # 학습 부분, caret패키지 함수로 해주세요!
    train.wknn <- train(gen~.,pro.train.normal, method = "kknn", trControl = cv,tuneGrid = wknn.grid)
    train.knn <- train(gen~., pro.train.normal, method = "knn", trControl = cv, tuneGrid = knn.grid)
    train.glm <- train(gen~., pro.train.normal, method = "glm", trControl = cv)
    train.lda <- train(gen~., pro.train.normal, method = "lda", trControl = repCv)
  # 예측 부분
    predict.wknn <- predict(train.wknn, pro.test.normal)
    predict.lda <- predict(train.lda, pro.test.normal)
    predict.glm <- predict(train.glm, pro.test.normal)
    predict.knn <- predict(train.knn, pro.test.normal)
  # predict count 부분
    df <- data.frame(model_knn=predict.knn, model_lda=predict.lda, model_glm=predict.glm, model_wknn=predict.wknn)
  # 최종 결과물 반환
    result <- NULL
    for (i in 1: nrow(df)){
        xtab <- table(t(df[i,]))
        rvalue <- names(sort(xtab, decreasing = T)[1])
        result <- c(result, rvalue)
    }
  return(as.factor(result))
}

voteClassifier1 <- function() {
  # 학습 부분, caret패키지 함수로 해주세요!
    train.wknn2 <- train(gen~.,pro.train.normal, method = "kknn", trControl = cv,tuneGrid = wknn.grid)
    train.knn2 <- train(gen~., pro.train.normal, method = "knn", trControl = cv, tuneGrid = knn.grid)
    train.lda2 <- train(gen~., pro.train.normal, method = "lda", trControl = repCv)
  # 예측 부분
    predict.wknn2 <- predict(train.wknn2, pro.test.normal)
    predict.lda2 <- predict(train.lda2, pro.test.normal)
    predict.knn2 <- predict(train.knn2, pro.test.normal)
  # predict count 부분
    df <- data.frame(model_knn=predict.knn2, model_lda=predict.lda2, model_wknn=predict.wknn2)
  # 최종 결과물 반환
    result <- NULL
    for (i in 1: nrow(df)){
        xtab <- table(t(df[i,]))
        rvalue <- names(sort(xtab, decreasing = T)[1])
        result <- c(result, rvalue)
    }
  return(as.factor(result))
}

predict.ensemble <- voteClassifier()
predict.ensemble1 <- voteClassifier1()

+ Fold1: kmax=1, distance=1, kernel=optimal 
- Fold1: kmax=1, distance=1, kernel=optimal 
+ Fold1: kmax=3, distance=1, kernel=optimal 
- Fold1: kmax=3, distance=1, kernel=optimal 
+ Fold1: kmax=5, distance=1, kernel=optimal 
- Fold1: kmax=5, distance=1, kernel=optimal 
+ Fold1: kmax=7, distance=1, kernel=optimal 
- Fold1: kmax=7, distance=1, kernel=optimal 
+ Fold1: kmax=9, distance=1, kernel=optimal 
- Fold1: kmax=9, distance=1, kernel=optimal 
+ Fold1: kmax=1, distance=2, kernel=optimal 
- Fold1: kmax=1, distance=2, kernel=optimal 
+ Fold1: kmax=3, distance=2, kernel=optimal 
- Fold1: kmax=3, distance=2, kernel=optimal 
+ Fold1: kmax=5, distance=2, kernel=optimal 
- Fold1: kmax=5, distance=2, kernel=optimal 
+ Fold1: kmax=7, distance=2, kernel=optimal 
- Fold1: kmax=7, distance=2, kernel=optimal 
+ Fold1: kmax=9, distance=2, kernel=optimal 
- Fold1: kmax=9, distance=2, kernel=optimal 
+ Fold2: kmax=1, distance=1, kernel=optimal 
- Fold2: kmax=1, distance=1, kernel=optimal 
+ Fold2: k

Selecting tuning parameters
Fitting kmax = 9, distance = 1, kernel = optimal on full training set
+ Fold1: k=1 
- Fold1: k=1 
+ Fold1: k=3 
- Fold1: k=3 
+ Fold1: k=5 
- Fold1: k=5 
+ Fold1: k=7 
- Fold1: k=7 
+ Fold1: k=9 
- Fold1: k=9 
+ Fold2: k=1 
- Fold2: k=1 
+ Fold2: k=3 
- Fold2: k=3 
+ Fold2: k=5 
- Fold2: k=5 
+ Fold2: k=7 
- Fold2: k=7 
+ Fold2: k=9 
- Fold2: k=9 
+ Fold3: k=1 
- Fold3: k=1 
+ Fold3: k=3 
- Fold3: k=3 
+ Fold3: k=5 
- Fold3: k=5 
+ Fold3: k=7 
- Fold3: k=7 
+ Fold3: k=9 
- Fold3: k=9 
+ Fold4: k=1 
- Fold4: k=1 
+ Fold4: k=3 
- Fold4: k=3 
+ Fold4: k=5 
- Fold4: k=5 
+ Fold4: k=7 
- Fold4: k=7 
+ Fold4: k=9 
- Fold4: k=9 
+ Fold5: k=1 
- Fold5: k=1 
+ Fold5: k=3 
- Fold5: k=3 
+ Fold5: k=5 
- Fold5: k=5 
+ Fold5: k=7 
- Fold5: k=7 
+ Fold5: k=9 
- Fold5: k=9 
Aggregating results
Selecting tuning parameters
Fitting k = 7 on full training set
+ Fold1.Rep1: parameter=none 
- Fold1.Rep1: parameter=none 
+ Fold2.Rep1: parameter=none 
- Fold2.Rep1: parameter=none 

In [62]:
predict.final = as.numeric(predict.ensemble)-1
predict.final1 = as.numeric(predict.ensemble1)-1

In [63]:
predict.final1

[1] 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0
[39] 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
[77] 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0

In [64]:

#### 최종 결과물 제출
names(predict.final) <- "gen"
write.csv(predict.final,'정하민all.csv',row.names =F)
names(predict.final1) <- "gen"
write.csv(predict.final1,'정하민all-glm.csv',row.names =F)
### 0 - man 1 - woman